In [1]:
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/open_ai/one-piece-episode-text-generation')

In [0]:
def get_file_content(filename):
  try:
    text = ''
    with open(filename, 'r') as f:
      text = f.read().lower()
    return text
  except:
    return ''

In [0]:
data_dir = 'data'
data = ''''''

for filename in os.listdir(data_dir):
  filepath = os.path.join(data_dir, filename)
  data += get_file_content(filepath) + ' '

In [6]:
len(data)

3467335

In [7]:
chars = sorted(list(set(data)))
chars

['\x06',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '~',
 '\x80',
 '\x82',
 '\x85',
 '\x86',
 '\x88',
 '\x89',
 '\x8c',
 '\x8d',
 '\x91',
 '\x93',
 '\x94',
 '\x96',
 '\x97',
 '\x98',
 '\x99',
 '\x9c',
 '\x9d',
 '\xa0',
 '¡',
 '¥',
 '¦',
 '¨',
 '©',
 '¬',
 '¯',
 '°',
 '´',
 'º',
 '¼',
 'à',
 'â',
 'ã',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ò',
 'ó',
 'ô',
 'ö',
 'û',
 'ü']

In [0]:
chars = chars[:65]

In [9]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
char_to_int

{'\x06': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 '*': 11,
 '+': 12,
 ',': 13,
 '-': 14,
 '.': 15,
 '/': 16,
 '0': 17,
 '1': 18,
 '2': 19,
 '3': 20,
 '4': 21,
 '5': 22,
 '6': 23,
 '7': 24,
 '8': 25,
 '9': 26,
 ':': 27,
 ';': 28,
 '=': 29,
 '>': 30,
 '?': 31,
 '@': 32,
 '[': 33,
 '\\': 34,
 ']': 35,
 '^': 36,
 '_': 37,
 '`': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [10]:
int_to_char = dict((i, c) for c, i in char_to_int.items())
int_to_char

{0: '\x06',
 1: ' ',
 2: '!',
 3: '"',
 4: '#',
 5: '$',
 6: '%',
 7: '&',
 8: "'",
 9: '(',
 10: ')',
 11: '*',
 12: '+',
 13: ',',
 14: '-',
 15: '.',
 16: '/',
 17: '0',
 18: '1',
 19: '2',
 20: '3',
 21: '4',
 22: '5',
 23: '6',
 24: '7',
 25: '8',
 26: '9',
 27: ':',
 28: ';',
 29: '=',
 30: '>',
 31: '?',
 32: '@',
 33: '[',
 34: '\\',
 35: ']',
 36: '^',
 37: '_',
 38: '`',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [0]:
data = [char for char in data if char in chars]

In [12]:
num_chars_in_data = len(data)
num_chars_in_vocab = len(chars)

print('Total number of characters in data =', num_chars_in_data)
print('Total number of characters in vocabulary =', num_chars_in_vocab)

Total number of characters in data = 3455602
Total number of characters in vocabulary = 65


In [13]:
sequence_length = 200
data_x = []
data_y = []

step_size = 10

for i in range(0, num_chars_in_data-sequence_length, step_size):
  data_x.append( [char_to_int[char] for char in data[i: i+sequence_length]] )
  data_y.append( char_to_int[data[i+sequence_length]] )
  
num_patterns = len(data_x)
print('Total Patterns =', num_patterns)

Total Patterns = 345541


In [0]:
def get_sentence_from_list_indexes(list_indexes):
  return ''.join([ int_to_char[index] for index in list_indexes ])

In [15]:
print(get_sentence_from_list_indexes(data_x[222]))
print(int_to_char[data_y[222]])

ing that name touch your lips will do that. i doubt that the friends you lost are safe. losing onejust one friend is usually worthy of regret. but when facing the shichibukai gecko no, in the face of 
o


In [0]:
X = np.reshape(data_x, (num_patterns, sequence_length, 1))
X = X / float(num_chars_in_vocab)

y = np_utils.to_categorical(data_y)

In [17]:
model = Sequential()
model.add( LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True) )
model.add( Dropout(0.2) )
model.add( LSTM(256) )
model.add( Dropout(0.2) )
model.add( Dense(y.shape[1], activation='softmax') )

W0808 13:40:37.458060 139851540424576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 13:40:37.496629 139851540424576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 13:40:37.504511 139851540424576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 13:40:37.854776 139851540424576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0808 13:40:37.865534 

In [18]:
model_filepath = 'weights_improvement-20-1.7271.hdf5'
model.load_weights(model_filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0808 13:40:38.695250 139851540424576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0808 13:40:39.021799 139851540424576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
model.load_weights

In [47]:
start = np.random.randint(0, len(data_x)-1)
pattern = data_x[start]

print('Seed:', get_sentence_from_list_indexes(pattern) )

Seed: just like that? anyway, since it's mochi maki, then they can't start anything without the mochi. that also means that the trust in our shop depends on this. hurry to the castle please! sign: for the c


In [48]:
len_story_to_generate = 2000

print( get_sentence_from_list_indexes(pattern) )

for i in range(len_story_to_generate):
  x = np.reshape(pattern, (1, len(pattern), 1))
  x = x / float(num_chars_in_vocab)
  prediction = model.predict(x, verbose=0)
  index = np.argmax(prediction)
  #index = np.random.choice(len(chars), p=prediction[0])
  result = int_to_char[index]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:]
print('\nDone')

just like that? anyway, since it's mochi maki, then they can't start anything without the mochi. that also means that the trust in our shop depends on this. hurry to the castle please! sign: for the c
ountry with the straw hat is the part of the sea whth the straw hat is the part of the straw hat is the part of the sea whth the straw hat is the part of the sea whth the straw hat is the part of the sea whth the straw hat is the part of the sea whth the st

KeyboardInterrupt: ignored